#### Imports

In [ ]:
import os
import sys
import json
import tifffile

In [ ]:
sys.path.append('..')

In [ ]:
from src.utils.image_utils import load_czi_images
from cell_tracking import get_masks_paths
from ms2_gene_expression import MS2GeneExpressionProcessor

#### Functions

#### Inputs

In [ ]:
czi_file_path = '/home/dafei/data/MS2/gRNA2_12.03.25-st-13-II---.czi'
seg_maps_dir = '/home/dafei/output/MS2/3d_cell_segmentation/gRNA2_12.03.25-st-13-II---/masks'
background_sub_ms2_channel = "/home/dafei/output/MS2/3d_cell_segmentation/gRNA2_12.03.25-st-13-II---/C1-gRNA2_12.03.25-st-13-II---_ms2_channel_background_sub.tif"
ms2_filtered_z_projection = "/home/dafei/output/MS2/3d_cell_segmentation/gRNA2_12.03.25-st-13-II---/SUM_C1-gRNA2_12.03.25-st-13-II---_ms2_channel_background_sub_z_project_sum_slice.tif"

In [ ]:
image_data = load_czi_images(czi_file_path)

In [ ]:
masks_paths = get_masks_paths(seg_maps_dir)

In [ ]:
ms2_background_sub = tifffile.imread(background_sub_ms2_channel)

In [ ]:
ms2_background_sub.shape

In [ ]:
ms2_z_projection = tifffile.imread(ms2_filtered_z_projection)

In [ ]:
tracklets_path = '/home/dafei/output/MS2/3d_cell_segmentation/gRNA2_12.03.25-st-13-II---/masks/tracklets_matching_iou.json'

In [ ]:
with open(tracklets_path, 'r') as f:
    tracklets = json.load(f)

In [ ]:
ids = list(tracklets.keys())

In [ ]:
processor = MS2GeneExpressionProcessor(tracklets, 6, image_data, masks_paths, ms2_z_projection)
processor.process()